# <center> Innoscape - Telecom Paristech - Projet fil rouge </center>
# <center> Mapping Groupes de produits </center>

In [17]:
# Librairies 
import pandas as pd
import numpy as np
from scipy import stats as scs
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import FrenchStemmer
import re
import random
from Preprocessing import *

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 60)

#chemin = "../Data/"
chemin = "C:/Users/JN/Desktop/Telecom PT/Projet fil rouge/github/INNOSCAPE/Data/"
chemin_val = "/Users/valentinphetchanpheng/Documents/MS BGD Telecom ParisTech/Projet Fil Rouge/"

## Importation des données et reformatage

In [3]:
# Importation des données vendeurs
df_seller_origin = pd.read_csv(chemin + "t3_all_fr_prices_posweekly_prod_without_price.csv", sep = ",")

#Création d'un df allégé pour le travail - on ne retient que les colonnes identifiées comme utiles pour l'analyse à ce stade
columns_to_keep = ['t3_all_fr_prices_posweekly_prod.productfamily_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory4_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory3_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory2_seller',
                   't3_all_fr_prices_posweekly_prod.prodcategory1_seller',
                   't3_all_fr_prices_posweekly_prod.productseller_name',
                   't3_all_fr_prices_posweekly_prod.id_source',
                   't3_all_fr_prices_posweekly_prod.brand_seller']

df_seller = df_seller_origin[columns_to_keep]

#on raccourcit un peu les noms des colonnes en ôtant les préfixes redondants
shortened_names = map(lambda string: string.replace('t3_all_fr_prices_posweekly_prod.',''), columns_to_keep)
col_name_dict = dict(zip(columns_to_keep, shortened_names))
df_seller.rename(columns=col_name_dict, inplace=True)

In [4]:
# Importation des données références catégories Innoscape
df_ref_origin = pd.read_csv(chemin + "2018.11.21_innoscape_internal_architecture.csv")
df_ref = df_ref_origin.drop("Date", axis=1)

#On garde que la colonne correspondante à la famille de produit de référence finale
columns_to_keep = ['innoscape_ref_productfamily_seller']
df_ref = df_ref[columns_to_keep]
df_ref = df_ref.drop_duplicates()

In [5]:
df_seller = df_seller.dropna(how='all', subset=['productfamily_seller',
                                                'prodcategory4_seller',
                                                'prodcategory3_seller',
                                                'prodcategory2_seller',
                                                'prodcategory1_seller',
                                                'productseller_name'])
df_seller = df_seller.fillna("")

# Retrait des caractères spéciaux et de la ponctuation
dictionary = {'à':'a','â':'a','ä':'a',
              'é':'e','è':'e','ê':'e','ë':'e',
              'ï':'i','î':'i',
              'ô':'o','ö':'o',
              'ù':'u','û':'u',
              'ç':'c',
              ',':' ', "'":" ", ";":" ", ":":" ", "\?":" ", "!":" ", "\.":'', "  ":" ", 
              "\(":"","\)":"","\[":"","\]":"","-":" ","\+":"","/":"","\*":"","–":" ",
              "&":"et"
              }


In [6]:
stop_words = {'a',
             'ai',
             'aie',
             'aient',
             'aies',
             'ait',
             'as',
             'au',
             'aura',
             'aurai',
             'auraient',
             'aurais',
             'aurait',
             'auras',
             'aurez',
             'auriez',
             'aurions',
             'aurons',
             'auront',
             'aux',
             'avaient',
             'avais',
             'avait',
             'avec',
             'avez',
             'aviez',
             'avions',
             'avons',
             'ayant',
             'ayante',
             'ayantes',
             'ayants',
             'ayez',
             'ayons',
             'c',
             'ce',
             'ces',
             'd',
             'dans',
             'de',
             'des',
             'du',
             'elle',
             'en',
             'es',
             'est',
             'et',
             'eu',
             'eue',
             'eues',
             'eurent',
             'eus',
             'eusse',
             'eussent',
             'eusses',
             'eussiez',
             'eussions',
             'eut',
             'eux',
             'eûmes',
             'eût',
             'eûtes',
             'furent',
             'fus',
             'fusse',
             'fussent',
             'fusses',
             'fussiez',
             'fussions',
             'fut',
             'fûmes',
             'fût',
             'fûtes',
             'il',
             'j',
             'je',
             'l',
             'la',
             'le',
             'leur',
             'lui',
             'm',
             'ma',
             'mais',
             'me',
             'mes',
             'moi',
             'mon',
             'même',
             'n',
             'ne',
             'nos',
             'notre',
             'nous',
             'on',
             'ont',
             'ou',
             'par',
             'pas',
             'pour',
             'qu',
             'que',
             'qui',
             's',
             'sa',
             'se',
             'sera',
             'serai',
             'seraient',
             'serais',
             'serait',
             'seras',
             'serez',
             'seriez',
             'serions',
             'serons',
             'seront',
             'ses',
             'soient',
             'sois',
             'soit',
             'sommes',
             'son',
             'sont',
             'soyez',
             'soyons',
             'suis',
             'sur',
             't',
             'ta',
             'te',
             'tes',
             'toi',
             'ton',
             'tu',
             'un',
             'une',
             'vos',
             'votre',
             'vous',
             'y',
             'à',
             'étaient',
             'étais',
             'était',
             'étant',
             'étante',
             'étantes',
             'étants',
             'étiez',
             'étions',
             'été',
             'étée',
             'étées',
             'étés',
             'êtes',
             ',',
             '(',
             ')',
             '[',
             ']',
             '-',
             '/',
             '+',
             '*',
             '{',
             '}',
             '!',
             ';',
             ':',
             '?',
             '.',
             '&'
             }


In [7]:
df_ref.columns

Index(['innoscape_ref_productfamily_seller'], dtype='object')

In [8]:
df_ref=preprocessing(df_ref,df_ref.columns,dictionary, stop_words)

list_new_col = ['productfamily_seller', 'prodcategory4_seller', 'prodcategory3_seller', 'prodcategory2_seller', 'prodcategory1_seller', 'productseller_name']
df_seller = preprocessing(df_seller, list_new_col, dictionary, stop_words)

In [9]:
df_seller.head(10)

productfamily_seller   prodcategory4_seller  \
0     Colles à carrelage     Colles à carrelage   
1                                                 
2                                                 
3                 Mastic                 Mastic   
4                  Cires                  Cires   
5                                                 
6      Perceuse-visseuse              Perceuses   
7  Produits d'étanchéité  Produits d'étanchéité   
8              Arroseurs              Arroseurs   
9                                                 

               prodcategory3_seller                      prodcategory2_seller  \
0                Colles et adhésifs                             Quincaillerie   
1                                                                               
2  Accessoires pour lave-vaisselles                               Accessoires   
3                Colles et adhésifs                             Quincaillerie   
4  Peintures, teintures et solvants  Peintures, outils et traitement des murs   
5                                    Peintures, outils et traitement des murs   
6         Outillage électroportatif       Outillage à main et électroportatif   
7                Colles et adhésifs                             Quincaillerie   
8               Matériel d'arrosage                                 Jardinage   
9                                                                               

  prodcategory1_seller                                 productseller_name  \
0            Bricolage  SikaCeram Xtra - Colle à carrelage intérieur e...   
1                       Ariston 3200362 scaldabagno elettrico PRO 100 H/3   
2  Gros électroménager  AEG Panier À Couverts Lave-vaisselle Cage & Ha...   
3            Bricolage              Colle-mastic pour joints d'étanchéité   
4            Bricolage  Sikagard Protection Sol MAT - Imperméabilisant...   
5            Bricolage  Sikagard Protection Toiture inclinée - Protect...   
6            Bricolage  Perceuse-visseuse RYOBI 18V OnePlus - 1 batter...   
7            Bricolage  Sika Multiseal - Bande d'étanchéité autocollan...   
8               Jardin            Black and Decker 34345 Arroseur Impulse   
9                       Ryobi-RXB01 Cortasetos-Écartement des dents La...   

  id_source      brand_seller productfamily_seller_formatted  \
0    AMZ_FR     SIKAFRANCESAS             colles a carrelage   
1    AMZ_FR  ARISTONTHERMOSPA                                  
2    AMZ_FR               AEG                                  
3    AMZ_FR     SIKAFRANCESAS                         mastic   
4    AMZ_FR     SIKAFRANCESAS                          cires   
5    AMZ_FR     SIKAFRANCESAS                                  
6    AMZ_FR             RYOBI              perceuse visseuse   
7    AMZ_FR     SIKAFRANCESAS          produits d etancheite   
8    AMZ_FR       BLACKDECKER                      arroseurs   
9    AMZ_FR             RYOBI                                  

  prodcategory4_seller_formatted    prodcategory3_seller_formatted  \
0             colles a carrelage                colles et adhesifs   
1                                                                    
2                                 accessoires pour lave vaisselles   
3                         mastic                colles et adhesifs   
4                          cires   peintures teintures et solvants   
5                                                                    
6                      perceuses         outillage electroportatif   
7          produits d etancheite                colles et adhesifs   
8                      arroseurs               materiel d arrosage   
9                                                                    

            prodcategory2_seller_formatted prodcategory1_seller_formatted  \
0                            quincaillerie                      bricolage   
1                                                                       

In [10]:
cols_to_keep = ['productfamily_seller',
                'prodcategory4_seller',
                'prodcategory3_seller',
                'prodcategory2_seller',
                'prodcategory1_seller',
                'productseller_name',
                'id_source',
                'productfamily_seller_stem',
                'prodcategory4_seller_stem',
                'prodcategory3_seller_stem',
                'prodcategory2_seller_stem',
                'prodcategory1_seller_stem',
                'productseller_name_stem']

df2 = df_seller[cols_to_keep]
df2.rename(columns={'productfamily_seller_stem':"prod_fam",
                'prodcategory4_seller_stem':'cat4',
                'prodcategory3_seller_stem':'cat3',
                'prodcategory2_seller_stem':'cat2',
                'prodcategory1_seller_stem':'cat1',
                'productseller_name_stem':'description'}, inplace=True)

In [11]:
df2.head()

productfamily_seller prodcategory4_seller              prodcategory3_seller  \
0   Colles à carrelage   Colles à carrelage                Colles et adhésifs   
1                                                                               
2                                            Accessoires pour lave-vaisselles   
3               Mastic               Mastic                Colles et adhésifs   
4                Cires                Cires  Peintures, teintures et solvants   

                       prodcategory2_seller prodcategory1_seller  \
0                             Quincaillerie            Bricolage   
1                                                                  
2                               Accessoires  Gros électroménager   
3                             Quincaillerie            Bricolage   
4  Peintures, outils et traitement des murs            Bricolage   

                                  productseller_name id_source  \
0  SikaCeram Xtra - Colle à carrelage intérieur e...    AMZ_FR   
1  Ariston 3200362 scaldabagno elettrico PRO 100 H/3    AMZ_FR   
2  AEG Panier À Couverts Lave-vaisselle Cage & Ha...    AMZ_FR   
3              Colle-mastic pour joints d'étanchéité    AMZ_FR   
4  Sikagard Protection Sol MAT - Imperméabilisant...    AMZ_FR   

           prod_fam              cat4                       cat3  \
0  {coll, carrelag}  {coll, carrelag}              {coll, adhes}   
1                {}                {}                         {}   
2                {}                {}  {accessoir, vaissel, lav}   
3          {mastic}          {mastic}              {coll, adhes}   
4             {cir}             {cir}   {teintur, solv, peintur}   

                           cat2                  cat1  \
0                 {quincailler}            {bricolag}   
1                            {}                    {}   
2                   {accessoir}  {gros, electromenag}   
3                 {quincailler}            {bricolag}   
4  {mur, outil, trait, peintur}            {bricolag}   

                                         description  
0  {blanc, interieur, carrelag, max, xtra, coll, ...  
1             {elettrico, ariston, pro, scaldabagno}  
2  {couvercl, compart, couvert, aeg, lav, vaissel...  
3                   {joint, mastic, coll, etancheit}  
4  {impermeabilis, protect, sol, sikagard, effet,...

## Mapping des familles de produits

In [12]:
df_test = df2
df_test.head()

productfamily_seller prodcategory4_seller              prodcategory3_seller  \
0   Colles à carrelage   Colles à carrelage                Colles et adhésifs   
1                                                                               
2                                            Accessoires pour lave-vaisselles   
3               Mastic               Mastic                Colles et adhésifs   
4                Cires                Cires  Peintures, teintures et solvants   

                       prodcategory2_seller prodcategory1_seller  \
0                             Quincaillerie            Bricolage   
1                                                                  
2                               Accessoires  Gros électroménager   
3                             Quincaillerie            Bricolage   
4  Peintures, outils et traitement des murs            Bricolage   

                                  productseller_name id_source  \
0  SikaCeram Xtra - Colle à carrelage intérieur e...    AMZ_FR   
1  Ariston 3200362 scaldabagno elettrico PRO 100 H/3    AMZ_FR   
2  AEG Panier À Couverts Lave-vaisselle Cage & Ha...    AMZ_FR   
3              Colle-mastic pour joints d'étanchéité    AMZ_FR   
4  Sikagard Protection Sol MAT - Imperméabilisant...    AMZ_FR   

           prod_fam              cat4                       cat3  \
0  {coll, carrelag}  {coll, carrelag}              {coll, adhes}   
1                {}                {}                         {}   
2                {}                {}  {accessoir, vaissel, lav}   
3          {mastic}          {mastic}              {coll, adhes}   
4             {cir}             {cir}   {teintur, solv, peintur}   

                           cat2                  cat1  \
0                 {quincailler}            {bricolag}   
1                            {}                    {}   
2                   {accessoir}  {gros, electromenag}   
3                 {quincailler}            {bricolag}   
4  {mur, outil, trait, peintur}            {bricolag}   

                                         description  
0  {blanc, interieur, carrelag, max, xtra, coll, ...  
1             {elettrico, ariston, pro, scaldabagno}  
2  {couvercl, compart, couvert, aeg, lav, vaissel...  
3                   {joint, mastic, coll, etancheit}  
4  {impermeabilis, protect, sol, sikagard, effet,...

On reprend le travail précédent, en se concentrant seulement sur les noms de familles dans un premier temps. On redéfinit pour cela les fonctions utilisées précédemment, pour factoriser le code:

In [13]:
df_test.columns

Index(['productfamily_seller', 'prodcategory4_seller', 'prodcategory3_seller',
       'prodcategory2_seller', 'prodcategory1_seller', 'productseller_name',
       'id_source', 'prod_fam', 'cat4', 'cat3', 'cat2', 'cat1', 'description'],
      dtype='object')

In [18]:
#on calcule ensuite, pour chaque famille de la df_reflogie Innoscape, et pour chaque produit de l'échantillon test,
#la similitude de Jaccard:

def jaccard(row_df1, col_df1, df2, idx_df2, col_df2):
    """ fonction à appliquer sur un dataframe, noté ici df1
    calcule la distance de Jaccard entre la colonne col_df1 du dataframe df1,
    et une certaine cellule située à l'index idx_df2, dans la colonne col_df2 d'un autre dataframe noté df2
    """
    return len(row_df1[col_df1]&df2.loc[idx_df2,col_df2]) / len(row_df1[col_df1]|df2.loc[idx_df2,col_df2])

def jaccard_matrix(df, col_df_tokenized, df_ref, col_df_ref, col_df_ref_tokenized, string_header):
    """ 
    prend un DataFrame (df) en input, et donne en output une copie de df avec une série de colonnes
    contenant les distances de jaccard avec chaque famille d'une df_reflogie produits (df_ref).
    
    df: DataFrame: base produits obtenue par crawling et préprocessée
    col_df_tokenized: string: nom de la colonne de df où se trouvent les données à comparer (tokenisées sous forme de sets de mots)
    df_ref: DataFrame: contient df_reflogie produits Innoscape
    col_df_ref: str: colonne de df_ref où se trouvent les données à comparer (texte brut)
    col_df_ref_tokenized: colonne de df_ref où se trouvent les données à comparer (tokenisées sous forme de sets de mots)
    string_header: préfixe à mettre dans les noms des colonnes ajoutées
    """
    
    df_copy = df.copy()
    for element in df_ref[col_df_ref].unique():
        col_title = str(string_header)+'_'+str(element)
        fam_dict = dict(zip(df_ref[col_df_ref], df_ref.index.values))
        df_copy[col_title] = df_copy.apply(jaccard,
                                           args=(col_df_tokenized, df_ref, fam_dict[element], col_df_ref_tokenized),
                                           axis=1)
    return df_copy

def get_max_similarity(row, string_header):
    """
    Donne le max des scores de jaccard, le nombre de catégories pour lesquelles ce score est atteint, 
    et les indices des catégories pour lesquelles ce score est atteint
    Inputs:
        row : ligne d'un dataframe
        strin_header : str :  préfixe des en-têtes des colonnes où sont stockés les scores de jaccard
    Outputs:
        maximum : float : max des scores de Jaccard
        count_max : int :  nb de fois où le score max est atteint
        count_max : indices des familles pour lesquelles le score max est atteint
    """
    scores = [row[string_header+'_'+str(element)] for element in df_ref.innoscape_ref_productfamily_seller.unique()]
    maximum = np.max(scores)
    count_max = scores.count(maximum)
    
    if maximum == 0.0:
        argmax = [-1]
    else:
        argmax = [i for i in range(len(scores)) if scores[i]==maximum]
        
    return [maximum, count_max, argmax]

# on crée un dictionnaire pour exprimer explicitement les correspondances trouvées en fonction des catégories Innoscape:
cat_dict = dict(zip(range(len(df_ref.innoscape_ref_productfamily_seller.unique())),df_ref.innoscape_ref_productfamily_seller.unique()))
cat_dict[-1] = "no match"

def unpack(row, col_name, idx):
    return row[col_name][idx]
    
def matching(df, col_df_tokenized):
    """
    Compare une colonne tokénisée d'un dataframe avec les familles produits Innoscape.
    Inputs:
        df : DataFrame contenant la colonne à comparer
        col_df_tokenized : str : titre de la colonne à comparer ; doit contenir des sets de mots
    
    Output : DataFrame contenant la famille Innoscape la plus proche au sens de Jaccard, avec le score
    et les ex-aequos éventuels
    """
    cols_to_keep = ['productfamily_seller',
                    'productseller_name',
                    'id_source',
                    'max_sim_'+col_df_tokenized,
                    'max_occurences_'+col_df_tokenized,
                    'best_match_'+col_df_tokenized,
                    'ex_aequos_'+col_df_tokenized,'prod_fam', 'cat4', 'cat3', 'cat2', 'cat1', 'description']
    
    df2 = jaccard_matrix(df, col_df_tokenized, df_ref,'innoscape_ref_productfamily_seller' , 'innoscape_ref_productfamily_stem', "similarity_"+col_df_tokenized)
    df2['max'] = df2.apply(get_max_similarity, args=(["similarity_"+col_df_tokenized]), axis=1)
    df2[['max_sim_'+col_df_tokenized, 'max_occurences_'+col_df_tokenized,'argmax']] = pd.DataFrame(df2['max'].values.tolist(), index= df2.index)
    df2['1st_match'] = df2.apply(unpack, args=('argmax',0), axis=1)
    df2['best_match_'+col_df_tokenized] = df2['1st_match'].apply(lambda key: cat_dict[key])
    df2['ex_aequos_'+col_df_tokenized] = df2['argmax'].apply(lambda array: [cat_dict[element] for element in array])
    
    return df2[cols_to_keep]

On reprend ensuite la démarche réalisée précédemment sur les noms de familles produits:

Essayons de segmenter cet échantillon en fonction de la valeur de max_jaccard:

In [ ]:
bucket0 = matching_prod_fam[matching_prod_fam['max_sim_prod_fam']==0].dropna()
bucket1 = matching_prod_fam.where((matching_prod_fam['max_sim_prod_fam']>0) & (matching_prod_fam['max_sim_prod_fam']<0.2)).dropna()
bucket2 = matching_prod_fam.where((matching_prod_fam['max_sim_prod_fam']>0.2) & (matching_prod_fam['max_sim_prod_fam']<0.4)).dropna()
bucket3 = matching_prod_fam.where((matching_prod_fam['max_sim_prod_fam']>0.4) & (matching_prod_fam['max_sim_prod_fam']<0.6)).dropna()
bucket4 = matching_prod_fam.where((matching_prod_fam['max_sim_prod_fam']>0.6) & (matching_prod_fam['max_sim_prod_fam']<0.8)).dropna()
bucket5 = matching_prod_fam.where((matching_prod_fam['max_sim_prod_fam']>0.8)).dropna()

buckets = [bucket0, bucket1, bucket2, bucket3, bucket4, bucket5]

Voyons, catégorie par catégorie, dans quelle mesure les regroupements sont pertinents:

In [ ]:
bucket5.head(40)

In [ ]:
bucket4.head(40)

In [ ]:
bucket3.head(40)

In [ ]:
bucket2.head(40)

On commence à observer quelques rapprochements erronés, par exemple "produits de traitement et accessoires piscine" avec "produits et traitement bois interieur" (distance de Jaccard de 0.33). On peut cependant envisager de faire baisser le nombre d'erreur en supprimant, au cours du preprocessing, des termes peu spécifiques (par exemple "produit" ou "électrique").

In [ ]:
bucket1.head(40)

On constate un certain nombre d'erreurs d'attribution, pour des similitudes de Jaccard inférieures à 0.15 environ. Dans la suite, on retiendra donc provisoirement ce seuil pour déclencher une deuxième vague de regroupements sur la base des descriptions produits. Voyons maintenant ce qui se passe quand on complète le rapprochement par noms de familles par le rapprochement à partir des descriptions de produits:

In [29]:
matching_descr = matching(df_test, "description")
matching_descr.head()

productfamily_seller  \
21130                Huile de démoulage   
20648                   Electroportatif   
25866  Enduits et revêtements de façade   
16496            Raccord pour gouttière   
27018                   Tuiles de verre   

                                      productseller_name  max_sim_description  \
21130                Lankodem Eco 341 bidon de 20 litres             0.000000   
20648  Visseuse a choc sans batterie M18 FID-0 - 3700...             0.222222   
25866  Enduit tyrolien régional weber.lenzit F - ton ...             0.071429   
16496                            Coude mf 87°30 O80 noir             0.000000   
27018  Tuile de verre Panne Bisch Genre Romane Méridi...             0.000000   

       max_occurences_description  \
21130                         105   
20648                           1   
25866                           1   
16496                         105   
27018                         105   

                                  best_match_description  \
21130                                           no match   
20648         Perceuse, perforateur et visseuse sans fil   
25866  Poudre béton, chaux, ciment, enduit, mortier, ...   
16496                                           no match   
27018                                           no match   

                                   ex_aequos_description  \
21130                                         [no match]   
20648       [Perceuse, perforateur et visseuse sans fil]   
25866  [Poudre béton, chaux, ciment, enduit, mortier,...   
16496                                         [no match]   
27018                                         [no match]   

                     prod_fam cat4                   cat3  \
21130        {huil, demoulag}   {}                     {}   
20648           {electroport}   {}                     {}   
25866  {facad, enduit, revet}   {}                     {}   
16496     {raccord, gouttier}   {}  {gouttier, accessoir}   
27018            {tuil, verr}   {}                     {}   

                             cat2                          cat1  \
21130               {gros, oeuvr}  {gros, voir, bpe, oeuvr, tp}   
20648               {electroport}       {quincailler, outillag}   
25866           {ite, mur, revet}            {ite, isol, platr}   
16496  {gouttier, bardag, toitur}             {sol, mur, revet}   
27018                      {tuil}                   {couvertur}   

                                             description  
21130                       {lankodem, bidon, litr, eco}  
20648           {san, fid, cpsmn, choc, batter, visseux}  
25866  {kg, enduit, regional, pierr, f, sac, weberlen...  
16496                             {°, mf, coud, o, noir}  
27018  {x, tuil, pann, roman, n°, bisch, genr, cm, me...

In [ ]:
bucket0 = matching_descr[matching_descr['max_sim_descr']==0].dropna()
bucket1 = matching_descr.where((matching_descr['max_sim_descr']>0) & (matching_descr['max_sim_descr']<0.05)).dropna()
bucket2 = matching_descr.where((matching_descr['max_sim_descr']>0.05) & (matching_descr['max_sim_descr']<0.1)).dropna()
bucket3 = matching_descr.where((matching_descr['max_sim_descr']>0.1) & (matching_descr['max_sim_descr']<0.15)).dropna()
bucket4 = matching_descr.where((matching_descr['max_sim_descr']>0.15) & (matching_descr['max_sim_descr']<0.2)).dropna()
bucket5 = matching_descr.where((matching_descr['max_sim_descr']>0.2)).dropna()

buckets = [bucket0, bucket1, bucket2, bucket3, bucket4, bucket5]

In [ ]:
bucket5.head(40)

In [ ]:
bucket4.head(40)

In [ ]:
bucket3.head(40)

On voit là encore que les rapprochements deviennent imprécis lorsque la similitude de Jaccard tombe en-dessous de 0.15.

Voyons maintenant ce qui se passe quand on utilise les catégories vendeur les plus profondes (catégorie 4):

In [30]:
matching_cat4 = matching(df_test, "cat4")
matching_cat4.head()

productfamily_seller  \
21130                Huile de démoulage   
20648                   Electroportatif   
25866  Enduits et revêtements de façade   
16496            Raccord pour gouttière   
27018                   Tuiles de verre   

                                      productseller_name  max_sim_cat4  \
21130                Lankodem Eco 341 bidon de 20 litres           0.0   
20648  Visseuse a choc sans batterie M18 FID-0 - 3700...           0.0   
25866  Enduit tyrolien régional weber.lenzit F - ton ...           0.0   
16496                            Coude mf 87°30 O80 noir           0.0   
27018  Tuile de verre Panne Bisch Genre Romane Méridi...           0.0   

       max_occurences_cat4 best_match_cat4 ex_aequos_cat4  \
21130                  105        no match     [no match]   
20648                  105        no match     [no match]   
25866                  105        no match     [no match]   
16496                  105        no match     [no match]   
27018                  105        no match     [no match]   

                     prod_fam cat4                   cat3  \
21130        {huil, demoulag}   {}                     {}   
20648           {electroport}   {}                     {}   
25866  {facad, enduit, revet}   {}                     {}   
16496     {raccord, gouttier}   {}  {gouttier, accessoir}   
27018            {tuil, verr}   {}                     {}   

                             cat2                          cat1  \
21130               {gros, oeuvr}  {gros, voir, bpe, oeuvr, tp}   
20648               {electroport}       {quincailler, outillag}   
25866           {ite, mur, revet}            {ite, isol, platr}   
16496  {gouttier, bardag, toitur}             {sol, mur, revet}   
27018                      {tuil}                   {couvertur}   

                                             description  
21130                       {lankodem, bidon, litr, eco}  
20648           {san, fid, cpsmn, choc, batter, visseux}  
25866  {kg, enduit, regional, pierr, f, sac, weberlen...  
16496                             {°, mf, coud, o, noir}  
27018  {x, tuil, pann, roman, n°, bisch, genr, cm, me...

In [ ]:
bucket0 = matching_cat4[matching_cat4['max_sim_cat4']==0].dropna()
bucket1 = matching_cat4.where((matching_cat4['max_sim_cat4']>0) & (matching_cat4['max_sim_cat4']<0.2)).dropna()
bucket2 = matching_cat4.where((matching_cat4['max_sim_cat4']>0.2) & (matching_cat4['max_sim_cat4']<0.4)).dropna()
bucket3 = matching_cat4.where((matching_cat4['max_sim_cat4']>0.4) & (matching_cat4['max_sim_cat4']<0.6)).dropna()
bucket4 = matching_cat4.where((matching_cat4['max_sim_cat4']>0.6) & (matching_cat4['max_sim_cat4']<0.8)).dropna()
bucket5 = matching_cat4.where((matching_cat4['max_sim_cat4']>0.8)).dropna()

buckets = [bucket0, bucket1, bucket2, bucket3, bucket4, bucket5]

In [ ]:
bucket1.head(40)

In [ ]:
bucket0.shape[0]

On observe des correspondances avec les catégories Innoscape dans près de 70% des cas. Cependant, les rapprochements sont moins précis qu'avec les familles produits. On peut envisager un seuil de similitude de 0.2-0.25. On observe par ailleurs que les rapprochements les plus pertinents semble être les suivants:
    1) rapprochement entre la famille produit et les familles produit Innoscape
    2) rapprochement entre la catégorie la plus profonde et les familles produit Innoscape
    3) rapprochement entre la description et les familles produit Innoscape
    
Quels résultats obtient-on en essayant successivement ces trois méthodes, et en retenant un rapprochement dès que la mesure de similitude dépasse le seuil voulu?

In [112]:
#on concatène les résultats obtenus par les trois méthodes dans un seul dataframe:
matching_prod_fam = matching(df_test, "prod_fam")
matching_descr = matching(df_test, "description")
matching_cat4 = matching(df_test, "cat4")

cols_to_drop = ['id_source','productfamily_seller', 'productseller_name','cat4', 'cat3', 'cat2', 'cat1']

matching_descr.drop(cols_to_drop, axis=1, inplace=True)
matching_cat4.drop(cols_to_drop, axis=1, inplace=True)
compare = pd.concat([matching_prod_fam, matching_descr, matching_cat4], axis=1)


In [81]:
compare.head()

productfamily_seller  \
29333          Electroportatif   
6110                    Tuiles   
16385      Girouette de jardin   
14969  Etanchéité des toitures   
729        Chariots d'arrosage   

                                      productseller_name id_source  \
29333  Pointe crantée électro-zinguée - Ø 2,8x75 mm -...     PP_FR   
6110                                     Chatière TB-10      CM_FR   
16385                Girouette en métal "Maestro" - Gris     MM_FR   
14969  Coin intérieur pour membrane d'étanchéité Easy...     LM_FR   
729                               Gardena Dévidoir Mural    AMZ_FR   

       max_sim_prod_fam  max_occurences_prod_fam  \
29333          0.250000                        1   
6110           0.000000                      105   
16385          0.333333                        2   
14969          0.250000                        1   
729            0.500000                        1   

                                     best_match_prod_fam  \
29333  Accessoires et consommables outillage électrop...   
6110                                            no match   
16385                                 Mobilier de jardin   
14969                           Mastic colle Etanchéité    
729                                             Arrosage   

                                      ex_aequos_prod_fam             prod_fam  \
29333  [Accessoires et consommables outillage électro...        {electroport}   
6110                                          [no match]               {tuil}   
16385              [Mobilier de jardin, Abris de jardin]    {jardin, girouet}   
14969                         [Mastic colle Etanchéité ]  {toitur, etancheit}   
729                                           [Arrosage]   {arrosag, chariot}   

                     cat4                 cat3                    cat2  \
29333                  {}                   {}           {electroport}   
6110                   {}                   {}                  {tuil}   
16385                  {}      {jardin, decor}  {jardin, mobili, jeux}   
14969                  {}  {toitur, etancheit}   {sol, mur, etancheit}   
729    {tuyal, accessoir}  {arrosag, materiel}              {jardinag}   

                            cat1  \
29333    {quincailler, outillag}   
6110   {tuil, accessoir, ardois}   
16385           {jardin, piscin}   
14969       {menuiser, material}   
729                     {jardin}   

                                             description  max_sim_description  \
29333  {crante, x, electro, pc, band, mm, point, ø, z...             0.000000   
6110                                       {chatier, tb}             0.000000   
16385                  {"maestro", girouet, metal, gris}             0.000000   
14969  {x, gris, etancheit, coin, easygum, membran, i...             0.111111   
729                            {mural, garden, devidoir}             0.000000   

       max_occurences_description    best_match_description  \
29333                         105                  no match   
6110                          105                  no match   
16385                         105                  no match   
14969                           1  Mastic colle Etanchéité    
729                           105                  no match   

            ex_aequos_description             prod_fam  \
29333                  [no match]        {electroport}   
6110                   [no match]               {tuil}   
16385                  [no match]    {jardin, girouet}   
14969  [Mastic colle Etanchéité ]  {toitur, etancheit}   
729                    [no match]   {arrosag, chariot}   

                                             description  max_sim_cat4  \
29333  {crante, x, electro, pc, band, mm, point, ø, z...      0.000000   
6110                                       {chatier, tb}      0.000000   
16385                  {"maestro", girouet, metal, gris}      0.000000   
14969  {x, gris, etancheit, coin, easygum, membran

In [107]:
def choose_match(row, threshold1, threshold2, threshold3):
    scores = row[['max_sim_prod_fam','max_sim_cat4','max_sim_descr']]
    matches = row[['best_match_prod_fam','best_match_cat4','best_match_descr']]
    
    if row['max_sim_prod_fam'] >= threshold1:
        return pd.Series([row['best_match_prod_fam'], row['max_sim_prod_fam']])
    elif row['max_sim_cat4'] >= threshold2:
        return pd.Series([row['best_match_cat4'], row['max_sim_cat4']])
    elif row['max_sim_description'] >= threshold3:
        return pd.Series([row['best_match_description'],row['max_sim_description']])
    elif max(scores) > np.float64(0) :
        return pd.Series([matches.values[np.argmax(scores.values)],max(scores.values)])
    else:
        return pd.Series(["no match", 0.0])

In [113]:
compare[['best_match','similarity']]=compare.apply(choose_match, args=(0.2,0.2,0.2), axis=1)

C:\Users\JN\Anaconda3\lib\site-packages\pandas\core\series.py:851: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


In [114]:
compare[['productfamily_seller', 'best_match_prod_fam', 'best_match_description', 'best_match_cat4', 'best_match', 'similarity']].head(50)

productfamily_seller  \
0                              Colles à carrelage   
1                                                   
2                                                   
3                                          Mastic   
4                                           Cires   
5                                                   
6                               Perceuse-visseuse   
7                           Produits d'étanchéité   
8                                       Arroseurs   
9                                                   
10                                                  
11                          Pompes pour fontaines   
12                           Sécateurs et Ciseaux   
13            Accessoires pour outillage sans fil   
14          Pièces et accessoires pour chaudières   
15                                                  
16             Filtres pour aspirateurs portatifs   
17                              Meuleuses d'angle   
18                                                  
19                                       Conduits   
20                                       Conduits   
21                                Mastic au latex   
22                             Pièces et raccords   
23                           Casques et écouteurs   
24                           Bordures pour jardin   
25                            Marteau perforateur   
26                                  Tronçonneuses   
27                                Stickers muraux   
28                             Colles à carrelage   
29                                                  
30                                                  
31                                Mastic au latex   
32                              Sacs porte-outils   
33                              Packs de batterie   
34                     Lampes de poche (standard)   
35                                       Conduits   
36                               Ciseaux de maçon   
37                                Mastic au latex   
38                                   Taille-haies   
39                                                  
40                                  Scies égoïnes   
41                          Produits d'étanchéité   
42                                                  
43                     Lampes de poche (standard)   
44                                  Tronçonneuses   
45                                                  
46                                                  
47       Aspirateurs balais et balais électriques   
48  Moulinets pêche au lancer léger et ultraléger   
49                             Colles à carrelage   

                          best_match_prod_fam  \
0                    Mastic colle Etanchéité    
1                                    no match   
2                                    no match   
3                    Mastic colle Etanchéité    
4                                    no match   
5                                    no match   
6   Perceuse, perforateur et visseuse filaire   
7                    Produits phytosanitaires   
8                                    no match   
9                                    no match   
10                                   no match   
11                                   no match   
12                 Cutter, scalpel et ciseaux   
13         Accessoires Outillage stationnaire   
14                                 Chaudière    
15                                   no match   
16          Aspirateurs, souffleurs, broyeurs   
17                     Meuleuse et rainureuse   
18                                   no match   
19          Conduit et accessoires d'aération   
20          Conduit et accessoires d'aération   
21                   Mastic colle Etanchéité    
22                                   no match   
23                                   no match   
24                         Mobilier de jardin   
25                                    Marteau   
26

In [116]:
positive_matches = compare[compare['similarity']>0.4]
positive_matches[['productfamily_seller', 'best_match_prod_fam', 'best_match_description', 'best_match_cat4', 'best_match', 'similarity']]

productfamily_seller  \
6                        Perceuse-visseuse   
25                     Marteau perforateur   
38                            Taille-haies   
59                                 Bobines   
60                          Pulvérisateurs   
65                            Taille-haies   
72                       Chauffe-eau à gaz   
76                    Pistolets d'arrosage   
79                       Perceuse-visseuse   
118                  Minuteries d'arrosage   
119                    Marteau perforateur   
124                      Chauffe-eau à gaz   
125                    Marteau perforateur   
129                    Marteau perforateur   
131    Outils multifonction et accessoires   
134                                Bobines   
146                                          
161                      Perceuse-visseuse   
165                            Accessoires   
166                              Chargeurs   
189                                          
203                      Perceuse-visseuse   
207                    Marteau perforateur   
213                           Taille-haies   
219                           Taille-haies   
231                              Chargeurs   
236                      Perceuse-visseuse   
248           Perches élagueuses manuelles   
249           Perches élagueuses manuelles   
261                      Perceuse-visseuse   
...                                    ...   
30191                  Mastics, étanchéité   
30196                  Mastics, étanchéité   
30226       Electroportatif - Consommables   
30238       Electroportatif - Consommables   
30248       Electroportatif - Consommables   
30312       Electroportatif - Consommables   
30337       Electroportatif - Consommables   
30377       Electroportatif - Consommables   
30382       Electroportatif - Consommables   
30395                  Mastics, étanchéité   
30444       Electroportatif - Consommables   
30475                  Mastics, étanchéité   
30479                  Mastics, étanchéité   
30502                  Mastics, étanchéité   
30506       Electroportatif - Consommables   
30528       Electroportatif - Consommables   
30549       Electroportatif - Consommables   
30567       Electroportatif - Consommables   
30569                  Mastics, étanchéité   
30584                  Mastics, étanchéité   
30598       Electroportatif - Consommables   
30613       Electroportatif - Consommables   
30624       Electroportatif - Consommables   
30634       Electroportatif - Consommables   
30648       Electroportatif - Consommables   
30676                  Mastics, étanchéité   
30698       Electroportatif - Consommables   
30714       Electroportatif - Consommables   
30845                               Mousse   
30848                               Mousse   

                                     best_match_prod_fam  \
6              Perceuse, perforateur et visseuse filaire   
25                                               Marteau   
38                                          Taille-haies   
59                                              no match   
60                                         Pulvérisation   
65                                          Taille-haies   
72                                       chauffe-eau gaz   
76                                              Arrosage   
79             Perceuse, perforateur et visseuse filaire   
118                                             Arrosage   
119                                              Marteau   
124                                      chauffe-eau gaz   
125                                              Marteau   
129                                              Marteau   
131                                Outils multifonctions   
134                                             no match   
146                                             no match   
161            Perceuse, perforateur et visseuse filaire   
165                          

In [117]:
positive_matches.shape

(6996, 27)

In [119]:
positive_matches[['id_source','productfamily_seller','best_match', 'similarity']].head(100)

id_source                 productfamily_seller  \
6      AMZ_FR                    Perceuse-visseuse   
25     AMZ_FR                  Marteau perforateur   
38     AMZ_FR                         Taille-haies   
59     AMZ_FR                              Bobines   
60     AMZ_FR                       Pulvérisateurs   
65     AMZ_FR                         Taille-haies   
72     AMZ_FR                    Chauffe-eau à gaz   
76     AMZ_FR                 Pistolets d'arrosage   
79     AMZ_FR                    Perceuse-visseuse   
118    AMZ_FR                Minuteries d'arrosage   
119    AMZ_FR                  Marteau perforateur   
124    AMZ_FR                    Chauffe-eau à gaz   
125    AMZ_FR                  Marteau perforateur   
129    AMZ_FR                  Marteau perforateur   
131    AMZ_FR  Outils multifonction et accessoires   
134    AMZ_FR                              Bobines   
146    AMZ_FR                                        
161    AMZ_FR                    Perceuse-visseuse   
165    AMZ_FR                          Accessoires   
166    AMZ_FR                            Chargeurs   
189    AMZ_FR                                        
203    AMZ_FR                    Perceuse-visseuse   
207    AMZ_FR                  Marteau perforateur   
213    AMZ_FR                         Taille-haies   
219    AMZ_FR                         Taille-haies   
231    AMZ_FR                            Chargeurs   
236    AMZ_FR                    Perceuse-visseuse   
248    AMZ_FR         Perches élagueuses manuelles   
249    AMZ_FR         Perches élagueuses manuelles   
261    AMZ_FR                    Perceuse-visseuse   
..        ...                                  ...   
535    AMZ_FR                Marteaux perforateurs   
545    AMZ_FR                Marteaux perforateurs   
556    AMZ_FR              Chauffe-eau électriques   
584    AMZ_FR                              Bobines   
586    AMZ_FR              Chauffe-eau électriques   
587    AMZ_FR                            Chargeurs   
606    AMZ_FR                          Accessoires   
610    AMZ_FR                    Perceuse-visseuse   
614    AMZ_FR                          Accessoires   
618    AMZ_FR                 Cloueur de charpente   
625    AMZ_FR                    Perceuse-visseuse   
631    AMZ_FR                 Pistolets d'arrosage   
647    AMZ_FR                              Bobines   
653    AMZ_FR                    Tuyaux d'arrosage   
660    AMZ_FR                                Têtes   
666    AMZ_FR                    Pompes d'arrosage   
673    AMZ_FR                 Pistolets d'arrosage   
691    AMZ_FR                          Accessoires   
694    AMZ_FR                         Taille-haies   
695    AMZ_FR                    Perceuse-visseuse   
703    AMZ_FR                                        
707    AMZ_FR                       Coupe-bordures   
722    AMZ_FR                              Bobines   
724    AMZ_FR                              Bobines   
726    AMZ_FR                Marteaux perforateurs   
729    AMZ_FR                  Chariots d'arrosage   
732    AMZ_FR                  Marteau perforateur   
747    AMZ_FR                    Perceuse-visseuse   
749    AMZ_FR                          Accessoires   
758    AMZ_FR                          Accessoires   

                                    best_match  similarity  
6    Perceuse, perforateur et visseuse filaire    0.500000  
25                                     Marteau    0.500000  
38                                Taille-haies    1.000000  
59          Débroussailleuse et coupe-bordures    0.500000  
60                               Pulvérisation    1.000000  
65                                Taille-haies    1.000000  
72                             chauffe-eau gaz    1.000000  
76                                    Arrosage    0.500000  
79   Perceuse, perforateur et visseuse filaire    0.500000  
118                                   Arrosag

In [120]:
stats_by_cat = positive_matches.groupby('best_match').count()

In [121]:
len(positive_matches.best_match.unique())

61

In [122]:
stats_by_cat['productfamily_seller']

best_match
Accessoires Gros oeuvre                                     10
Accessoires Outillage stationnaire                           1
Accessoires chauffe-eau                                    387
Accessoires couverture                                      12
Accessoires d'isolation                                      1
Accessoires et consommables outillage électroportatif      599
Accessoires et divers outils peintre                         1
Accessoires pour nettoyeur haute pression                    9
Accessoires pour outillage à main de jardin                  5
Ampoules et leds                                            11
Anti-mousse                                                 85
Arrosage                                                    35
Aspirateurs, souffleurs, broyeurs                           44
Aérateur et VMC                                            157
Barbecue                                                     1
Bardage                                     

In [105]:
missing_cats = [c for c in df_ref['innoscape_ref_productfamily_seller'].unique() if c not in positive_matches['best_match'].unique()]

In [106]:
missing_cats

['Accessoires Gros oeuvre',
 'Divers Gros oeuvre',
 'Adoucisseur',
 'Accessoires Outillage stationnaire',
 'Outillage stationnaire',
 'Aspirateurs, Nettoyeurs HP et accessoires',
 'Compresseurs et accessoires',
 'Chauffe-eau thermodynamique',
 'Pistolet à colle et agrafeuse',
 'Divers OEP',
 'Taillanderie',
 'Outillage à main',
 'Produits phytosanitaires',
 'Purificateur, déshumidificateur et humidificateur ',
 'Carré potager et table de rempotage',
 'Protection jardin, paillage, tuteurs et accessoires',
 'Bardage ',
 'Charpente ',
 "Accessoires d'isolation ",
 'Chaudière ',
 'Peinture cuisine et bain',
 'Accessoires et divers outils peintre',
 "Accessoires amenagement et produits d'entretien ",
 'Disjoncteur de branchement',
 'Mesure et tracage',
 'Accessoires plein air',
 'Electroménager, four, hotte, réfrigérateur et plaque de cuisson',
 'Accessoires pour electroménager-four-hotte-réfrigérateur-plaque de cuisson',
 'Contenants',
 'Accessoires du tableau électrique (bornier, peigne,e